# Studio Definitivo 
#### Filippo Iacobelli e Luca Rossicone

Lo scopo di questa parte conclusiva del progetto è stata quella di realizzare algoritmi che sfruttano la GPU per parallelizzare e ottenere prestazioni ancora migliori.
I vari esperimenti li abbiamo eseguiti sul superserver NVIDIA DGX-1, del Laboratorio di Scienze Computazionali.

In [ ]:
using CUDA
using LinearAlgebraicRepresentation
using BenchmarkTools
using DataStructures
Lar = LinearAlgebraicRepresentation

Abbiamo provato ad eseguire diversi esperimenti cercando di sfruttare approcci di tipo diverso. Per far si che le differenze sulle prestazioni fossero più visibili abbiamo aumentato considerevolmente le quantità dei segmenti che costituiscono il dominio.

In [ ]:
function circle(radius=1., angle=2*pi)
    function circle0(shape=[3600])
        V, EV = Lar.cuboidGrid(shape)
        V = (angle/shape[1])*V
        V = hcat(map(u->[radius*cos(u); radius*sin(u)], V)...)
        W, EW = Lar.simplifyCells(V, EV)
        return W, EW
    end
    return circle0
end

In questo tipo tentativo abbiamo sfruttato i CuArrays per eseguire i prodotti fra vettori e una semplice funzione che viene chiamata da map per eseguire il prodotto fra gli elementi del vettore e il raggio e per il calcolo di seni e coseni.

In [ ]:
function circle_map(u,radius)
    [radius*cos(u); radius*sin(u)]
end

In [ ]:
function circle_cuda(radius=1., angl=2*pi)
    function circle0(shape=[3600])
        V, EV = Lar.cuboidGrid(shape)
        V = CUDA.@sync V.*angl/shape[1]
        V = CUDA.@sync map(x -> circle_map(x,radius), V)
        V = hcat(V...)
        W, EW = Lar.simplifyCells(V, EV)
        return W, EW
    end
    return circle0
end

Questo secondo tentativo tenta di sfruttare dei kernel per il prodotto del dominio col raggio che vengono lanciati su GPU, in particolare su 256 threads differenti. 

In [ ]:
function gpu_mul!(y, x)
    index = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    stride = gridDim().x * blockDim().x
    for i = index:stride:length(y)
        @inbounds y[i] *= x[i]
    end
    return
end

In [ ]:
function circle_cuda_slow(radius=1.5, angl=2*pi)
    function circle0(shape=[3600])
        len = shape[1]+1
        numblocks = ceil(Int, shape[1]+1/256)
        V, EV = Lar.cuboidGrid(shape)
        V = CuArray(V)
        r = CUDA.fill(angl/shape[1], shape[1]+1) 
        @cuda threads=256 blocks=numblocks gpu_mul!(V,r)
        W = sin.(V)
        V = cos.(V)
        fill!(r,1)
        @cuda threads=256 blocks=numblocks gpu_mul!(V,r)
        @cuda threads=256 blocks=numblocks gpu_mul!(W,r)
        V = vcat(V,W)
        W, EW = Lar.simplifyCells(V, EV)
        return W, EW
        end
    return circle0
end

In [ ]:
@btime circle()()
@btime circle_cuda_slow()()
@btime circle_cuda()()


162.281 ms (188455 allocations: 11.44 MiB)

5.164 s (687549 allocations: 38.73 MiB)

112.335 ms (188460 allocations: 11.47 MiB)


Altri esperimenti sono stati eseguiti con la funzione `toroidal` la cui esecuzione è decisamente più onerosa rispetto a `circle`.

In [ ]:
function toroidal(r=1., R=2., angle1=2*pi, angle2=2*pi)
    function toroidal0(shape=[240, 360])
        V, CV = Lar.simplexGrid(shape)
        V = [angle1/(shape[1]) 0;0 angle2/(shape[2])]*V
        W = [V[:, k] for k=1:size(V, 2)]
        V = hcat(map(p->let(u, v)=p;[(R+r*cos(u))*cos(v);
          (R+r*cos(u))*sin(v);-r*sin(u)]end, W)...)
        W, CW = Lar.simplifyCells(V, CV)
        return W, CW
    end
    return toroidal0
  end

Abbiamo nuovamente tentato di portare i domini della figura su GPU in modo da eseguire i calcoli mediante CuArrays, per poi tornare su CPU solamente per l'esecuzione di `simplifyCells`.

In [ ]:
function toroidal_cuda(r=1., R=2., angle1=2*pi, angle2=2*pi)
    function toroidal0(shape=[240, 360])
        V, CV = Lar.simplexGrid(shape)
        V = CuArray(V)
        C = CuArray([angle1/(shape[1]) 0;0 angle2/(shape[2])])
        V = collect(eachrow(C*V))
        u = first(V); z = last(V)
        sinU = sin.(u); sinZ = sin.(z)
        cosU = cos.(u); cosZ = cos.(z)
        tmp = cosU.*r.+R
        v = hcat(tmp.*cosZ, tmp.*sinZ, -r*sinU)
        V = Array{Float32}(undef, length(u), 3)
        copyto!(V,v)
        W, CW = Lar.simplifyCells(V', CV)
        return W, CW
      end
    return toroidal0
  end

In [ ]:
@btime toroidal()()
@btime toroidal_cuda()()

8.430 s (4584863 allocations: 695.77 MiB)

1.102 s (7163959 allocations: 790.69 MiB)

[*Link al repository del progetto*](https://bitbucket.org/zoso9999/mapper.jl/src/main/)